<a href="https://colab.research.google.com/github/DrAlexSanz/NLP-SPEC-C3/blob/main/W2/Perplexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Working with JAX numpy and calculating perplexity: Ungraded Lecture Notebook
Normally you would import numpy and rename it as np.

However in this week's assignment you will notice that this convention has been changed.

Now standard numpy is not renamed and trax.fastmath.numpy is renamed as np.

The rationale behind this change is that you will be using Trax's numpy (which is compatible with JAX) far more often. Trax's numpy supports most of the same functions as the regular numpy so the change won't be noticeable in most cases.

In [1]:
!pip install sentencepiece==0.1.91
!pip install trax

import trax.fastmath.numpy as np

import trax

import numpy

     |████████████████████████████████| 1.1MB 13.4MB/s 
     |████████████████████████████████| 471kB 11.8MB/s 
     |████████████████████████████████| 2.6MB 54.5MB/s 
     |████████████████████████████████| 174kB 51.9MB/s 
     |████████████████████████████████| 3.7MB 52.7MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 1.4MB 60.2MB/s 
     |████████████████████████████████| 348kB 50.8MB/s 
     |████████████████████████████████| 890kB 47.5MB/s 
     |████████████████████████████████| 2.9MB 31.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=02e8feb859c2c7e93ad41825c44bccda8847c197f8e0b46e9669187897c0ef50
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


### Apparently the seed doesn't need to be set in trax supervised... it's now in the training loop. This thing is still half baked.

In [2]:
# Setting random seeds
#trax.supervised.trainer_lib.init_random_number_generators(32)
numpy.random.seed(32)

In [3]:
numpy_array = numpy.random.random((5,10))
print(f"The regular numpy array looks like this:\n\n {numpy_array}\n")
print(f"It is of type: {type(numpy_array)}")

The regular numpy array looks like this:

 [[0.85888927 0.37271115 0.55512878 0.95565655 0.7366696  0.81620514
  0.10108656 0.92848807 0.60910917 0.59655344]
 [0.09178413 0.34518624 0.66275252 0.44171349 0.55148779 0.70371249
  0.58940123 0.04993276 0.56179184 0.76635847]
 [0.91090833 0.09290995 0.90252139 0.46096041 0.45201847 0.99942549
  0.16242374 0.70937058 0.16062408 0.81077677]
 [0.03514717 0.53488673 0.16650012 0.30841038 0.04506241 0.23857613
  0.67483453 0.78238275 0.69520163 0.32895445]
 [0.49403187 0.52412136 0.29854125 0.46310814 0.98478429 0.50113492
  0.39807245 0.72790532 0.86333097 0.02616954]]

It is of type: <class 'numpy.ndarray'>


In [4]:
trax_numpy_array = np.array(numpy_array)
print(f"The trax numpy array looks like this:\n\n {trax_numpy_array}\n")
print(f"It is of type: {type(trax_numpy_array)}")

The trax numpy array looks like this:

 [[0.8588893  0.37271115 0.55512875 0.9556565  0.7366696  0.81620514
  0.10108656 0.9284881  0.60910916 0.59655344]
 [0.09178413 0.34518623 0.6627525  0.44171348 0.5514878  0.70371246
  0.58940125 0.04993276 0.56179184 0.7663585 ]
 [0.91090834 0.09290995 0.9025214  0.46096042 0.45201847 0.9994255
  0.16242374 0.7093706  0.16062407 0.81077677]
 [0.03514718 0.5348867  0.16650012 0.30841038 0.04506241 0.23857613
  0.67483455 0.7823827  0.69520164 0.32895446]
 [0.49403188 0.52412134 0.29854125 0.46310815 0.9847843  0.50113493
  0.39807245 0.72790533 0.86333096 0.02616954]]

It is of type: <class 'jax.interpreters.xla.DeviceArray'>


# Calculating Perplexity
The perplexity is a metric that measures how well a probability model predicts a sample and it is commonly used to evaluate language models. It is defined as:

$$P(W) = \sqrt[N]{\prod_{i=1}^{N} \frac{1}{P(w_i| w_1,...,w_{n-1})}}$$
As an implementation hack, you would usually take the log of that formula (to enable us to use the log probabilities we get as output of our RNN, convert exponents to products, and products into sums which makes computations less complicated and computationally more efficient). You should also take care of the padding, since you do not want to include the padding when calculating the perplexity (because we do not want to have a perplexity measure artificially good). The algebra behind this process is explained next:

$$log P(W) = {log\big(\sqrt[N]{\prod_{i=1}^{N} \frac{1}{P(w_i| w_1,...,w_{n-1})}}\big)}$$$$ = {log\big({\prod_{i=1}^{N} \frac{1}{P(w_i| w_1,...,w_{n-1})}}\big)^{\frac{1}{N}}}$$
$$ = {log\big({\prod_{i=1}^{N}{P(w_i| w_1,...,w_{n-1})}}\big)^{-\frac{1}{N}}} $$$$ = -\frac{1}{N}{log\big({\prod_{i=1}^{N}{P(w_i| w_1,...,w_{n-1})}}\big)} $$$$ = -\frac{1}{N}{\big({\sum_{i=1}^{N}{logP(w_i| w_1,...,w_{n-1})}}\big)} $$

You will be working with a real example from this week's assignment. The example is made up of:

* Predictions : batch of tensors corresponding to lines of text predicted by the model.
* Targets : batch of actual tensors corresponding to lines of text.

In [5]:
from trax import layers as tl
!wget https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C3/main/W2/utils.py

!wget https://github.com/DrAlexSanz/NLP-SPEC-C3/raw/main/W2/predictions.npy
!wget https://github.com/DrAlexSanz/NLP-SPEC-C3/raw/main/W2/targets.npy

--2020-12-01 21:13:07--  https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C3/main/W2/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 970 [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]     970  --.-KB/s    in 0s      

2020-12-01 21:13:08 (62.5 MB/s) - ‘utils.py’ saved [970/970]

--2020-12-01 21:13:08--  https://github.com/DrAlexSanz/NLP-SPEC-C3/raw/main/W2/predictions.npy
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/DrAlexSanz/NLP-SPEC-C3/main/W2/predictions.npy [following]
--2020-12-01 21:13:08--  https://raw.githubusercontent.com/DrAlexSanz/NLP-S

In [7]:
predictions = numpy.load("predictions.npy")
targets = numpy.load("targets.npy")

#np is a jax devicearray
predictions = np.array(predictions)
targets = np.array(targets)

print(f"Predictions has shape {predictions.shape}")
print(f"Targets has shape {targets.shape}")

Predictions has shape (32, 64, 256)
Targets has shape (32, 64)


Notice that the predictions have an extra dimension with the same length as the size of the vocabulary used.

Because of this you will need a way of reshaping targets to match this shape. For this you can use trax.layers.one_hot().

Notice that predictions.shape[-1] will return the size of the last dimension of predictions.

In [11]:
reshaped_targets = tl.one_hot(targets, predictions.shape[-1])
print(f"Reshaped targets have shape {reshaped_targets.shape}")

Reshaped targets have shape (32, 64, 256)


By calculating the product of the predictions and the reshaped targets and summing across the last dimension, the total log perplexity can be computed:

In [12]:
total_log_perp = np.sum(predictions * reshaped_targets, axis = -1)

Now you will need to account for the padding so this metric is not artificially deflated (since a lower perplexity means a better model). For identifying which elements are padding and which are not, you can use np.equal() and get a tensor with 1s in the positions of actual values and 0s where there are paddings.

In [13]:
non_pad = 1.0 - np.equal(targets, 0)
print(f"Non pad has shape {non_pad.shape}")
print(non_pad)

Non pad has shape (32, 64)
[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]]


In [14]:
real_log_perp = non_pad * total_log_perp
print(f"Real log perp has shape {real_log_perp.shape}")
print(real_log_perp) #Notice the 0s

Real log perp has shape (32, 64)
[[ -5.396545    -1.0311184   -0.66916656 ...  -0.          -0.
   -0.        ]
 [ -4.5857706   -1.1341286   -8.538033   ...  -0.          -0.
   -0.        ]
 [ -5.2223887   -1.2824144   -0.17312431 ...  -0.          -0.
   -0.        ]
 ...
 [ -5.396545   -17.291681    -4.360766   ...  -0.          -0.
   -0.        ]
 [ -5.9313164  -14.247417    -0.2637329  ...  -0.          -0.
   -0.        ]
 [ -5.670536    -0.10595131   0.         ...  -0.          -0.
   -0.        ]]


To get a single average log perplexity across all the elements in the batch you can sum across both dimensions and divide by the number of elements. Notice that the result will be the negative of the real log perplexity of the model:

In [16]:
log_perp = -1 * np.sum(real_log_perp)/np.sum(non_pad)

print(f"The log perplexity of the model is {log_perp:.4f} and the perplexity is {np.exp(log_perp):.4f}")

The log perplexity of the model is 2.3281 and the perplexity is 10.2586
